# Modelling probability of admission to specialty, if admitted

This notebook demonstrates the second stage of prediction, to generate a probability of admission to a specialty for each patient in the ED if they are admitted. 

Here consult sequences provide the input to prediction, and the model is trained only on visits by adult patients that ended in admission. Patients less than 18 at the time of arrival to the ED are assumed to be admitted to paediatric wards. This assumption could be relaxed by changing the training data to include children, and changing how the inference stage is done. 

This approach assumes that, if admitted, a patient's probability of admission to any particular specialty is independent of their probability of admission to hospital. 

## Set up the notebook environment

In [7]:
# Reload functions every time
%load_ext autoreload 
%autoreload 2

In [2]:
from pathlib import Path
import sys
import json
import pandas as pd

root = Path().resolve().parent

sys.path.append(str(root/ 'src' / 'patientflow'))




## Load parameters

These are set in config.json. You can change these for your own purposes. But the times of day will need to match those in the provided dataset if you want to run this notebook successfully.

In [3]:
# indicate whether the notebook is being run locally for UCLH or with public datasets
uclh = False
from load import set_file_paths
from load import load_config_file


# set file location
if uclh:
    data_file_path, media_file_path, model_file_path, config_path = set_file_paths(
        train_dttm = None, data_folder_name = 'data-uclh', uclh = uclh, from_notebook=True, inference_time=False
    )
else:
     data_file_path, media_file_path, model_file_path, config_path = set_file_paths(
        train_dttm = None, data_folder_name = 'data-public', uclh = uclh, from_notebook=True, inference_time=False
    )   


# load params
params = load_config_file(config_path)

prediction_times = params["prediction_times"]
start_training_set, start_validation_set, start_test_set, end_test_set = params["start_training_set"], params["start_validation_set"], params["start_test_set"], params["end_test_set"]
x1, y1, x2, y2 = params["x1"], params["y1"], params["x2"], params["y2"]
prediction_window = params["prediction_window"]
epsilon = float(params["epsilon"])
yta_time_interval = params["yta_time_interval"]

## Load data

In [4]:
import pandas as pd
from load import set_data_file_names
from load import data_from_csv

if uclh:
    # data_path = USER_ROOT / 'ed-predictor' / 'data-raw'
    # model_file_path = USER_ROOT / 'ed-predictor' / 'trained-models'
    visits_path, visits_csv_path, yta_path, yta_csv_path = set_data_file_names(uclh, data_file_path, config_path)
else:
    visits_csv_path, yta_csv_path = set_data_file_names(uclh, data_file_path)

visits = data_from_csv(visits_csv_path, index_column = 'snapshot_id',
                            sort_columns = ["visit_number", "snapshot_date", "prediction_time"], 
                            eval_columns = ["prediction_time", "consultation_sequence", "final_sequence"])


visits['snapshot_date'] = pd.to_datetime(visits['snapshot_date']).dt.date


Select only the visits that ended in admission, and the relevant subset of columns

## Train the model

This is the function that trains the specialty model, loaded from a file. Below we will break it down step-by-step.

In [12]:
from train import train_specialty_model, get_default_visits
??train_specialty_model

Signature:
train_specialty_model(
    visits,
    model_name,
    model_metadata,
    model_file_path,
    uclh,
)
Docstring: <no docstring>
Source:   
def train_specialty_model(visits, model_name, model_metadata, model_file_path, uclh):
    # Select one snapshot per visit
    visits_single = select_one_snapshot_per_visit(visits, visit_col="visit_number")

    # Prepare dataset of admitted visits only for training specialty model
    admitted = visits_single[
        (visits_single.is_admitted) & ~(visits_single.specialty.isnull())
    ]
    filtered_admitted = get_default_visits(admitted, uclh=uclh)

    # convert consults data format from list to tuple (required input for SequencePredictor)
    filtered_admitted.loc[:, "consultation_sequence"] = filtered_admitted[
        "consultation_sequence"
    ].apply(lambda x: tuple(x) if x else ())
    filtered_admitted.loc[:, "final_sequence"] = filtered_admitted[
        "final_sequence"
    ].apply(lambda x: tuple(x) if x else ())

    # T

The first step in the function above is to handle the fact that there are multiple snapshots per visit and we only want one for each visit in the training set. 

In [13]:
from prepare import select_one_snapshot_per_visit

visits_single = select_one_snapshot_per_visit(visits, visit_col = 'visit_number')

print(visits.shape)
print(visits_single.shape)

(79814, 69)
(64497, 68)


To train the specialty model, we only use a subset of the columns. Here we can see the relevant columns 

In [15]:
if uclh:
    visits_single[['consultation_sequence', 'final_sequence', 'specialty', 'is_admitted', 'age_on_arrival']]
else:
    visits_single[['consultation_sequence', 'final_sequence', 'specialty', 'is_admitted', 'age_group']]


We filter down to only include admitted patients, and remove any with a null value for the specialty column, since this is the model aims to predict. 

In [16]:
admitted = visits_single[
    (visits_single.is_admitted) & ~(visits_single.specialty.isnull())
]

A function called `get_default_visits` handles the next step. This uses a function defined in `prepare.py`, called `create_special_category_objects` to identify any visit characteristics that will be handled differently when predicting specialty. At UCLH, we don't use a model for patients under 18; instead at UCLH we assume that all under 18s will be admitted to a paediatric specialty. Their visits are therefore not relevant for model training, and we remove them here. 

In [17]:
filtered_admitted = get_default_visits(admitted, uclh=uclh)
print(visits_single.shape)
print(filtered_admitted.shape)

(64497, 68)
(8020, 68)


The consultation sequence (which is captured at the snapshot) and the final sequence have been loaded from CSV, and need to be converted to tuples. 

In [18]:
# convert consults data format from list to tuple (required input for SequencePredictor)
filtered_admitted.loc[:, "consultation_sequence"] = filtered_admitted[
    "consultation_sequence"
].apply(lambda x: tuple(x) if x else ())
filtered_admitted.loc[:, "final_sequence"] = filtered_admitted[
    "final_sequence"
].apply(lambda x: tuple(x) if x else ())

Note that some visits that ended in admission had no consult request at the time they were sampled, as we can see below, where visits have an empty tuple

In [20]:
if uclh:
    filtered_admitted[['consultation_sequence', 'final_sequence', 'specialty', 'is_admitted', 'age_on_arrival']]
else:
    filtered_admitted[['consultation_sequence', 'final_sequence', 'specialty', 'is_admitted', 'age_group']]
    


The UCLH data (not shared publicly) includes more detailed data on consult type, as shown below. The public data has been simplified to a higher level (identified in the mapping below as 'type'). 

In [21]:
root

PosixPath('/Users/zellaking/Repos/patientflow')

In [22]:
model_input_path = Path(root / 'src' /  'patientflow'/ 'model-input')
name_mapping = pd.read_csv(str(model_input_path) + '/consults-mapping.csv')
name_mapping

,id,code,name,type
0,1,CON124,Inpatient consult to Neuro Ophthalmology,neuro
1,2,CON9,Inpatient consult to Neurology,neuro
2,3,CON34,Inpatient consult to Dietetics (N&D) - Not TPN,allied
3,4,CON134,Inpatient consult to PERRT,icu
4,5,CON163,IP Consult to MCC Complementary Therapy Team,pain
...,...,...,...,...
111,112,CON77,Inpatient consult to Paediatric Allergy,paeds
112,113,CON168,Inpatient consult to Acute Oncology Service,haem_onc
113,114,CON84,Inpatient consult to Paediatric Hematology - C...,haem_onc
114,115,CON122,Inpatient consult to Paediatric Epilepsy Service,paeds


In [23]:
name_mapping[name_mapping.code == 'CON157']

,id,code,name,type
14,15,CON157,Inpatient consult to Acute Medicine,acute


## Separate into training, validation and test sets

As part of preparing the data, each visit has already been allocated into one of three sets - training, vaidation and test sets. 


In [24]:
train_visits = filtered_admitted.loc[filtered_admitted.index.isin(filtered_admitted[filtered_admitted.training_validation_test == 'train'].index)]
valid_visits = filtered_admitted.loc[filtered_admitted.index.isin(filtered_admitted[filtered_admitted.training_validation_test == 'valid'].index)]
test_visits = filtered_admitted.loc[filtered_admitted.index.isin(filtered_admitted[filtered_admitted.training_validation_test == 'test'].index)]


assert train_visits.snapshot_date.min() == start_training_set
assert train_visits.snapshot_date.max() < start_validation_set
assert valid_visits.snapshot_date.min() == start_validation_set
assert valid_visits.snapshot_date.max() < start_test_set
assert test_visits.snapshot_date.min() == start_test_set
assert test_visits.snapshot_date.max() < end_test_set

## Train the model

Here, we load the SequencePredictor(), a function that takes a sequence as input (in this case consultation_sequence), a grouping variable (in this case final_sequence) and a outcome variable (in this case specialty), and uses a grouping variable to create a rooted directed tree. Each new consult in the sequence is a branching node of the tree. The grouping variable, final sequence, serves as the terminal nodes of the tree. The function maps the probability of each part-complete sequence of consults ending (via each final_sequence) in each specialty of admission.

In [25]:
from predictors.sequence_predictor import SequencePredictor

In [26]:
spec_model = SequencePredictor(
    input_var="consultation_sequence",
    grouping_var="final_sequence",
    outcome_var="specialty",
)
spec_model.fit(train_visits)



SequencePredictor(grouping_var='final_sequence',
                  input_var='consultation_sequence', outcome_var='specialty')

Passing an empty tuple to the trained model shows the probability of ending in each specialty, if a visit has had no consults yet. 

In [27]:
spec_model.predict(tuple()) 

{'surgical': 0.2699213233810772,
 'medical': 0.6304216259834579,
 'haem/onc': 0.099657050635465}

The probabilities for each consult sequence ending in a given observed specialty have been saved in the model. These can be accessed as follows: 

In [28]:
weights = spec_model.weights
print("For a visit which has one consult to acute medicine at the time of a snapsnot, the probabilities of ending up under a medical, surgical or haem/onc specialty are shown below")
if uclh:
    print({k: round(v, 3) for k, v in weights[tuple(['CON157'])].items()})
else:
    print({k: round(v, 3) for k, v in weights[tuple(['acute'])].items()})


For a visit which has one consult to acute medicine at the time of a snapsnot, the probabilities of ending up under a medical, surgical or haem/onc specialty are shown below
{'surgical': 0.013, 'medical': 0.948, 'haem/onc': 0.039}


The intermediate mapping of consultation_sequence to final_sequence can be accessed from the trained model like this. The first row shows the probability of a null sequence (ie no consults yet) ending in any of the final_sequence options. 

In [29]:
spec_model.input_to_grouping_probs

final_sequence,(),"(acute,)","(acute, acute)","(acute, acute, medical)","(acute, acute, medical, surgical)","(acute, acute, mental_health)","(acute, acute, palliative)","(acute, acute, surgical)","(acute, allied)","(acute, allied, acute)",...,"(surgical, surgical)","(surgical, surgical, acute)","(surgical, surgical, acute, mental_health, discharge, discharge)","(surgical, surgical, acute, surgical)","(surgical, surgical, icu)","(surgical, surgical, medical)","(surgical, surgical, obs_gyn)","(surgical, surgical, other)","(surgical, surgical, surgical)",probability_of_grouping_sequence
consultation_sequence,,,,,,,,,,,,,,,,,,,,,
(),0.009819,0.458837,0.013218,0.000755,0.000378,0.000755,0.000000,0.000378,0.005665,0.000378,...,0.007553,0.000755,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000378,0.534194
"(acute,)",0.000000,0.830409,0.005848,0.000000,0.000000,0.000000,0.000000,0.000000,0.014620,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.206980
"(acute, acute)",0.000000,0.000000,0.909091,0.045455,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.004438
"(acute, allied)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000202
"(acute, ambulatory)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(surgical, medical)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000403
"(surgical, obs_gyn)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000403
"(surgical, surgical)",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.814815,0.000000,0.037037,0.0,0.0,0.037037,0.037037,0.037037,0.037037,0.005447


In [31]:
from joblib import dump, load

MODEL__ED_SPECIALTY__NAME = 'ed_specialty'

# use this name in the path for saving the model
full_path = model_file_path / MODEL__ED_SPECIALTY__NAME 
full_path = full_path.with_suffix('.joblib')

# save the model
dump(spec_model, full_path)

['/Users/zellaking/Repos/patientflow/trained-models/ed_specialty.joblib']